In [1]:
import os
import polars as pl
import json

In [2]:
m_dir_path = "C:/Users/aronn/OneDrive/Asztali gép/Rajk/Prog 2/c5_moves_with_valid_id"
moves_files = os.listdir(m_dir_path)
g_path = "C:/Users/aronn/OneDrive/Asztali gép/Rajk/Prog 2/games.parquet"

castling_moves = ["O-O", "O-O-O", "O-O+", "O-O-O+", "O-O#", "O-O-O#",]

castle_wins_dict = {}

In [3]:
games = (
    pl.scan_parquet(g_path)
    .select(["gameId", "White", "Black", "Result"])
    .with_columns(
        [
            (
                pl.when(pl.col("Result") == "1-0")
                .then(pl.col("White"))
                .otherwise(pl.col("Black"))
                .alias("Winner")
            )
        ]
    )
    .drop(["White", "Black"])
)

In [4]:
for file in moves_files:
    print(f"Processing {file}")
    
    m_path = os.path.join(m_dir_path, file)

    moves = (
        pl.scan_parquet(m_path)
        .select(["gameId", "moveNo", "move"])
        .filter(pl.col("move").is_in(castling_moves))
        .join(games, on="gameId", how="inner")
        # filter moves where winner castled
        .filter(
            ((pl.col("moveNo").str.contains("[^.].$")) &
            (pl.col("Result") == "1-0")) |
            (pl.col("moveNo").str.contains("\.{3}$")) &
            (pl.col("Result") == "0-1")
            )
        .group_by("Winner")
        .count()
        .collect()
    )

    # update dict
    for row in moves.iter_rows():
        winner = row[0]
        n_wins = row[1]

        if winner in castle_wins_dict:
            castle_wins_dict[winner] += n_wins
        else:
            castle_wins_dict[winner] = n_wins

<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
C:\Users\aronn\AppData\Local\Temp\ipykernel_7696\1576031557.py:15: SyntaxWarning: invalid escape sequence '\.'
  (pl.col("moveNo").str.contains("\.{3}$")) &
C:\Users\aronn\AppData\Local\Temp\ipykernel_7696\1576031557.py:19: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  .count()


Processing file_0.parquet
Processing file_01.parquet
Processing file_02.parquet
Processing file_03.parquet
Processing file_04.parquet
Processing file_05.parquet
Processing file_06.parquet
Processing file_07.parquet
Processing file_08.parquet
Processing file_09.parquet
Processing file_10.parquet
Processing file_11.parquet
Processing file_12.parquet
Processing file_13.parquet
Processing file_14.parquet
Processing file_15.parquet
Processing file_16.parquet
Processing file_17.parquet
Processing file_18.parquet
Processing file_19.parquet
Processing file_20.parquet
Processing file_21.parquet
Processing file_22.parquet
Processing file_23.parquet
Processing file_24.parquet
Processing file_25.parquet
Processing file_26.parquet
Processing file_27.parquet
Processing file_28.parquet
Processing file_29.parquet
Processing file_30.parquet
Processing file_31.parquet
Processing file_32.parquet
Processing file_33.parquet
Processing file_34.parquet
Processing file_35.parquet
Processing file_36.parquet
Pr

In [7]:
sorted_dict = dict(sorted(castle_wins_dict.items(), key=lambda item: item[1], reverse=True))

with open("castle_wins_100.json", "w") as json_file:
    json.dump(dict(list(sorted_dict.items())[:100]), json_file, indent=4)

In [6]:
sorted_dict

{'kon_stan': 4465,
 'Kingscrusher-YouTube': 3468,
 'Josedeco': 3400,
 'Amybel95': 3292,
 'SemyonTsybinPhila': 3269,
 'bkan80': 3187,
 'Bykov_Misha': 3178,
 'joninoloco01': 3139,
 'lnaveran': 3116,
 'Traumaturge': 3077,
 'markjones25': 3051,
 'wateenellende': 2975,
 'dietmar_bratsche': 2797,
 'Andabarian': 2779,
 'bingser': 2777,
 'WoodAri': 2736,
 'azjargalaltangerel': 2625,
 'Hadi54': 2581,
 'merlinsr': 2578,
 'Mikelins': 2523,
 'batboldb': 2484,
 'luxaflex': 2469,
 'Roman_Kutsenko': 2467,
 'Alex-31': 2465,
 'pyp0614': 2426,
 'Brandts': 2421,
 'lolote2002': 2384,
 'Arshinov': 2377,
 'Nextcustomer': 2367,
 'rmnp': 2364,
 'Petrovich38': 2325,
 'Sito_Galur': 2291,
 'Sphob23': 2288,
 'KingArtur69': 2270,
 'Anignoshtev': 2267,
 'lebedev1966': 2243,
 'hindutime': 2216,
 'will_23': 2212,
 'Silv3r_Surf3r': 2208,
 'Kazibek2002': 2204,
 'jclb': 2196,
 'cavallnegre2000': 2195,
 'Vladd6566': 2159,
 'ilnamco': 2151,
 'Rubendor': 2149,
 'honda155': 2146,
 'budala11': 2136,
 'JJTTRR': 2108,
 'Eliass

In [55]:
moves

Winner,count
str,u32
"""burningvalcan""",3
"""Robin2569""",7
"""PrathyushaYell…",7
"""Kajan_kd7""",10
"""Minusmann""",7
"""Alovlu""",42
"""ShevchenkoM150…",6
"""KBurehman96""",1
"""Natale53""",1


In [46]:
moves#["Result"].n_unique()

gameId,moveNo,move,Result,Winner
str,str,str,str,str
"""rmJ9Ryr7""","""6...""","""O-O""","""0-1""","""KventinKorvaly…"
"""Wqah1euv""","""20...""","""O-O-O""","""0-1""","""antoniobo62"""
"""wzBwmHls""","""15.""","""O-O-O""","""1-0""","""apt-iva"""
"""xKoCdtEm""","""15.""","""O-O""","""0-1""","""sam-tex"""
"""xtsXDmo0""","""7...""","""O-O""","""0-1""","""Luisangel_Flor…"
"""RcVIIUDT""","""12.""","""O-O-O""","""1-0""","""Pepau"""
"""QFOpU8Do""","""7.""","""O-O""","""1-0""","""IorioC"""
"""HAbAfOn1""","""14...""","""O-O-O""","""0-1""","""Zohreh56"""
"""HAbAfOn1""","""12.""","""O-O""","""0-1""","""Zohreh56"""
